# Create a Website-Map with all linked Subdomains

The crawler visits every subdomain of a website once and stores their connections to each other.
Aferwards these connections can be visualised with the program gephi.

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
from random import shuffle
import pandas as pd

In [2]:
url = 'https://www.via-ev.de'
domain = '.'+url.split('.')[1]

driver = webdriver.Firefox()
driver.get(url)

html = driver.page_source
bs = BeautifulSoup(html, 'lxml')

In [3]:
def processLinks(bs, sub_url):
    for link in bs.findAll('a'):
        if 'href' in link.attrs:
            link = link['href']
            if link=='/':
                link = url
            if domain in link or 'http' not in link:
                if '#' not in link and not link.startswith('mail') and '.jpg' not in link and '.png' not in link and '.jpeg' not in link and '.svg' not in link:
                    link = link if domain in link else url+'/'+link if not link.startswith('/') else link[1:]
                    link = link if not link.endswith('/') else link[:-1]
                    edges.append([sub_url, link])
                    if link not in visited_urls and link not in urls_to_visit and '.pdf' not in link:
                        urls_to_visit.append(link)

In [4]:
edges = []
visited_urls = set()
urls_to_visit = []

visited_urls.add(url)
processLinks(bs, url)
        
while urls_to_visit:
    try:
        shuffle(urls_to_visit)
        sub_url = urls_to_visit.pop()
        driver.get(sub_url)
        visited_urls.add(sub_url)
    
        html = driver.page_source
        bs = BeautifulSoup(html, 'lxml')
        
        processLinks(bs, sub_url)
        
        #print('Pending: {}'.format(len(urls_to_visit)))
    except:
        print('Fehler: {}'.format(sub_url))

driver.close()

Fehler: leistungen/referenzen
Fehler: leistungen/referenzen
Fehler: leistungen/referenzen


In [5]:
df = pd.DataFrame(edges, columns=['Source', 'Target'])
df = df.drop_duplicates()

source_links = df['Source'].value_counts().to_frame().to_dict()
target_links = df['Target'].value_counts().to_frame().to_dict()

df['Source_Links'] = df['Source'].apply(lambda x: source_links['Source'][x])
df['Target_Links'] = df['Source'].apply(lambda x: target_links['Target'][x])

df.to_csv('build/'+domain[1:]+'_Website_Map_Edges.csv', index=False)
print(df.shape)
df.head()

(9517, 4)


,Source,Target,Source_Links,Target_Links
0,https://www.via-ev.de,https://www.via-ev.de,59,172
1,https://www.via-ev.de,https://www.via-ev.de/via,59,172
2,https://www.via-ev.de,https://www.via-ev.de/via/organisation,59,172
3,https://www.via-ev.de,https://www.via-ev.de/kontakt/impressum/vorstand,59,172
4,https://www.via-ev.de,https://www.via-ev.de/via/ressorts,59,172
